In [ ]:
import boto3
from datetime import datetime, timezone
import botocore
import os
import time
from pprint import pp
import json
import re

while True:
    today = datetime.now(timezone.utc)
    def get_last_modified(obj): return int(obj['LastModified'].strftime('%s'))

    s3 = boto3.client('s3')
    objs = s3.list_objects_v2(Bucket='raw-sprint-3')['Contents']
    last_added = [obj['Key']
                  for obj in sorted(objs, key=get_last_modified)][-1]
    BUCKET_NAME = 'raw-sprint-3'  # replace with your bucket name
    KEY = last_added  # replace with your object key
    s3 = boto3.resource('s3')
    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, last_added)
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise
    with open(last_added, "r") as f:
        file_content = f.read()
    regex = re.findall(r'"Body":.*?{[^{}]+}', file_content)

    json_data = []
    for line in regex:
        json_data.append(json.loads(line[7:]))

    with open(last_added, "w") as f:
        f.write(json.dumps(json_data))
    s3.meta.client.upload_file(last_added, 'stagged-sprint-3', last_added)
    os.remove(last_added)

    time.sleep(15)
